In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

#### Data Augmentation

For each image in the training set, create four shifted copies (shifted 1 pixel for each direction: left, right, up, down) and add to the training set in order to improve model performance.

In [3]:
X_train = train.drop(columns='label').to_numpy()
y_train = train['label'].to_numpy()

print(X_train.shape)
print(y_train.shape)

(42000, 784)
(42000,)


In [4]:
from scipy.ndimage import shift

def shift_img(img, dx, dy):
    return shift(img.reshape((28, 28)), [dy, dx]).reshape([-1])

In [5]:
X_aug = []
y_aug = []

for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
    for img, lab in zip(X_train, y_train):
        X_aug.append(shift_img(img, dx, dy))
        y_aug.append(lab)

X_train_new = np.append(X_train, X_aug, axis=0)
y_train_new = np.append(y_train, y_aug)

print(X_train_new.shape)
print(y_train_new.shape)

(210000, 784)
(210000,)


In [6]:
shuffle_idx = np.random.permutation(len(X_train))
X_train = X_train[shuffle_idx]
y_train = y_train[shuffle_idx]

#### KNN Classifier with Grid Search

Since grid search will take much longer to complete using the new augmented training data set, first perform grid search on the original training set, and use the best hyperparameters to fit a model using the augmented training set.

In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

param_grid = [{'n_neighbors': [3, 4, 5], 'weights': ["uniform", "distance"]}]

knn = KNeighborsClassifier()
gs = GridSearchCV(knn, param_grid, cv=5)
gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid=[{'n_neighbors': [3, 4, 5],
                          'weights': ['uniform', 'distance']}])

In [8]:
print(gs.best_params_)
print(gs.best_score_)

{'n_neighbors': 4, 'weights': 'distance'}
0.9695476190476191


In [9]:
knn = KNeighborsClassifier(**gs.best_params_)
knn.fit(X_train_new, y_train_new)

KNeighborsClassifier(n_neighbors=4, weights='distance')

In [10]:
X_test = test.to_numpy()
predictions = knn.predict(X_test)

In [11]:
output = pd.DataFrame({'ImageId': range(1, len(test) + 1), 'Label': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
